# Why is harmonisation needed?
This notebook will give a brief example of how differences arise from using different scanners.
If we were to acquire the same individual on twod ifferent days, we would expect the resutling brain
images to be nearly the same. However, if we were to acquire the subject on one scanner the first day
and another scanner the second day, we would see differences, as MRI scanning on its own is not inherently
desgned to be quantative.

In [1]:
import os
import matplotlib
%matplotlib widget

# Grabbing data with datalad
[Datalad](https://www.datalad.org/) is a git and git-annex based method of version control for data management, data sharing and reproducible science. For this lesson, it offers us a convenient way of grabbing the data that we need for the lesson. These comamands import datald so that we can use it to clone the repository into our virtual machine.

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import datalad.api as dl

In [4]:
dl_source='https://github.com/HealthBioscienceIDEAS/demon-imaging-data.git'
sample=dl.clone(dl_source,path='/tmp/sample',description='Cloned sample dataset for import')
sample.update(merge=True)
sample.siblings(action='enable',name='demons-data')

[INFO] Fetching updates for Dataset(/tmp/sample) 


.: demons-data(?) [git]


[{'action': 'enable-sibling',
  'path': '/tmp/sample',
  'type': 'sibling',
  'name': 'demons-data',
  'refds': '/tmp/sample',
  'status': 'ok'}]

In [5]:
bl_img=sample.get('./baseline_t1.nii.gz')

In [6]:
fu_img=sample.get('./followup_t1.nii.gz')

# Running imaging with Nipype
The python package [nipype](https://nipype.readthedocs.io/en/latest/index.html) provides an effective means in Python to run your image processing workflows, taking "building blocks" of individual commands, allowing you to piece together bits of various software packages (FSL, SPM, FreeSurfer, AFNI, MRtrix, etc) into complete pipelines. We are just going to use it for simple cases here by registereing our baseline and followup image.

In [7]:
from nipype.interfaces import niftyreg


In [8]:
node = niftyreg.RegAladin()
node.inputs.ref_file = bl_img[0]['path']
node.inputs.flo_file = fu_img[0]['path']
out_path=os.path.dirname(bl_img[0]['path'])
node.inputs.aff_file = os.path.join(out_path,'followup_to_baseline_aff.txt')
node.inputs.res_file = os.path.join(out_path,'followup_resampled.nii.gz')
node.cmdline

'reg_aladin -aff /tmp/sample/followup_to_baseline_aff.txt -flo /tmp/sample/followup_t1.nii.gz -omp 1 -ref /tmp/sample/baseline_t1.nii.gz -res /tmp/sample/followup_resampled.nii.gz'

In [9]:
node.run()

210909-14:14:00,12 nipype.interface INFO:
	 stdout 2021-09-09T14:14:00.011548:[NiftyReg WARNING] NiftyReg has not been compiled with OpenMP, the '-omp' flag is ignored
210909-14:14:00,15 nipype.interface INFO:
	 stdout 2021-09-09T14:14:00.011548:[reg_aladin] 
210909-14:14:00,16 nipype.interface INFO:
	 stdout 2021-09-09T14:14:00.011548:[reg_aladin] Command line:
210909-14:14:00,16 nipype.interface INFO:
	 stdout 2021-09-09T14:14:00.011548:[reg_aladin] 	 reg_aladin -aff /tmp/sample/followup_to_baseline_aff.txt -flo /tmp/sample/followup_t1.nii.gz -omp 1 -ref /tmp/sample/baseline_t1.nii.gz -res /tmp/sample/followup_resampled.nii.gz
210909-14:14:00,18 nipype.interface INFO:
	 stdout 2021-09-09T14:14:00.011548:[reg_aladin] 
210909-14:14:00,18 nipype.interface INFO:
	 stdout 2021-09-09T14:14:00.011548:[reg_aladin_sym] Parameters
210909-14:14:00,19 nipype.interface INFO:
	 stdout 2021-09-09T14:14:00.011548:[reg_aladin_sym] Platform: cpu_platform
210909-14:14:00,20 nipype.interface INFO:
	 std

## Checking the registration

Let's see if how well the registration did. 

In [10]:
import nanslice.jupyter as ns

Next, we initialise FSLeyes and create the frame, or window that we will be starting the viewer in.

In [16]:
base=ns.three_plane(bl_img[0]['path'], title='Baseline',interactive=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Then we grab the image data that is needed for viewing. We first create an O

In [12]:
from fsleyes.views.orthopanel import OrthoPanel
from fsl.data.image import Image
ortho = frame.addViewPanel(OrthoPanel)

In [18]:
overlayList.append(Image(bl_img[0]['path']))
overlayList.append(Image(fu_img[0]['path']))

In [19]:
frame.Close()
fsleyes.shutdown()